In [1]:
import json
import ipywidgets as widgets
import os
import time

# Define some helper functions to save and load json files

In [13]:
# set up the helper routines to load the jsons
# written by J Wallin

def jsonFromFile(fname):
    f = open(fname,"r")
    s = f.read()
    f.close()
    return json.loads(s)

def jsonToFile(fname):
    print(fname)
    
def listFiles(assetPath, printFiles):
    finalList = []
    fullList = os.listdir(assetPath)
    for f in fullList:
        if f.find("json") == len(f)-4:
            print(f)
            finalList.append(f)
    return finalList

def loadFileIntoDictionary(path, flist):
    jdata = {}
    for f in flist:
        jdata[f] = jsonFromFile(path + f)
    return jdata



def writeScene(myScene, verbose):
    # save it to an output file
    outputFile = myScene["jsonFileName"]

    print(os.path.isfile(outputPath + outputFile))

    # if there is a file already there
    print(outputPath + outputFile)
    fileExists = os.path.isfile(outputPath + outputFile)

    if verbose:
        while (fileExists or outputFile == "" ):
    
            print("This is a listing of files in this path: ")
            flist = os.listdir(outputPath)
            for f in flist:
                print(f)
            yn = input("The file " + outputPath + outputFile +" already exists \n Do you want to overwrite it? ")
            if yn == "y" or yn == "Y":
                fileExists = False
            else:
                outputFile = input("What is the new name for our file? ")
                fileExists = os.path.isfile(outputPath + outputFile)
                

    print("\nWriting " + outputPath+outputFile + "\n\n")


    # form the final scene as a string
    myScene["jsonFileName"] = outputFile
    finalJson = json.dumps(myScene, indent=4)
    f = open(outputPath + outputFile, "w", encoding='utf-8')
    f.write(json.dumps(myScene, ensure_ascii=False, indent=4))
    f.close()

    
def createScene(moduleDescription, sceneData, olist, verbose=False):
    sceneData["objects"] = olist
    
    
    # make a copy of the generic scene for the activity file
    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
    myScene.update(moduleDescription)
    myScene.update(sceneData)


    # we can dump the scene to see what it looks like so far  
    if verbose:
        print("This is the Scene\n\n\n")
        print(json.dumps(myScene, indent=4))    
    
    writeScene(myScene, verbose)    
    
    
# this is a code stub for merging two activity files together
def concatenateScenes( outputPath, outputFile, sceneList):

    print(outputPath, outputFile)
    # open the new file
    f = open(outputPath + outputFile, 'w', encoding='utf-8')
    
    for i in range(len(sceneList)):
        print("  \n scene")
        print(i)
        currentScene = sceneList[i]
        print(currentScene)
        
        # read in the files
        fn1 = outputPath + currentScene
        s1 = jsonFromFile(fn1)

        # Dump the file - but do NOT use any indents.  This will flatten
        # the file into a single line.  We then write it out as a line with a newline character.
        a1 = json.dumps(s1, ensure_ascii=False)
        f.write(a1 + "\n")

    # close it
    f.close()


# Load all the generic definition files that contain our base classes

In [3]:
# define the path to access the generic templates and the output files
assetPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/genericDefinitions/"     
outputPath = "C:/Users/jfwal/OneDrive/Documents/GitHub/cyberAR/\_Code Device/AR Labs/Assets/Resources/jsonDefinitions/"  
assetPath = "genericDefinitions/"
outputPath = "./"


# get the list of all the files in the generic directory and load them into a dictionary
flist = listFiles(assetPath,  True)
jdata = loadFileIntoDictionary(assetPath, flist)


# this prints out the assets so you can review them
showFiles = False
if (showFiles):
    for f in flist:
        print("===== file name" + f)
        print("---------\n",f, "\n\n\n", json.dumps(jdata[f], indent=4))
        
#print(json.dumps(jdata["genericActivity.json"]))

genericActivity.json
genericObject.json
genericPointerReceiver.json
genericRigidBody.json
genericScriptSimpleRotation.json


# Create the Recipe for our the objects and the general activity module

### Define the objects in the activity module

In [92]:
# variables for the objects

orbitalScale = 1.0
timeRate = 1.0
earthSize = 0.08
earthRotationTime = 2.0
moonOrbitalPeriod = 56.0
moonOffSetDistance = 2.4
moonSize = earthSize * 0.25


############
scriptData1 = {
    "name": "simpleRotation",
    "timeRate": timeRate,
    "rotationTime": earthRotationTime
}
script1 = json.dumps(scriptData1) 


############
############    
componentsToModify= {
                        "RigidBody": 
                        {
                            "isKinematic": True,
                            "useGravity": False
                        },
                        "PointerReceiver": 
                        {
                        }
                    }

############


earthPosition = { "x":0.0, 
                 "y": 0.0, 
                 "z":1.5}
earthScale = {"x": earthSize,
            "y": earthSize,
            "z": earthSize }

earth = {
                "type": "Prefabs/moveableSphere",
                "position": earthPosition,
                "scale": earthScale,
                "name": "Earth", 
                "texture": "Textures/2k_earth_daymap",
                "componentsToAdd": [script1]
            }


# make a copy of the generic dictionary for objects
earthObject = jdata['genericObject.json'].copy()

# add the generic component attributes
earthObject.update(jdata['genericRigidBody.json'])
earthObject.update(jdata['genericPointerReceiver.json'])
earthObject.update(earth)

# override the generic component attributes
for c in componentsToModify.keys():
    earthObject[c].update(componentsToModify[c])
               
############


moonOffset = {"x":moonOffSetDistance, "y":0.0, "z":0.0}
moonPosition = {"x": earthPosition['x'] + moonOffset['x'],
                "y": earthPosition['y'] + moonOffset['y'],
                "z": earthPosition['z'] + moonOffset['z']
               }

scriptData2 = {
    "name": "simpleOrbit",
    "moonPosition": moonPosition,
    "orbitalPeriod": moonOrbitalPeriod,
    "timeRate": timeRate,
    "orbitalScale": orbitalScale,
    "synchronousRotation":True
    }
script2 = json.dumps(scriptData2)
    
    
moon = {
                "type": "Prefabs/moveableSphere",
                "parent": "Earth",
                "position": moonPosition,
                "scale": {"x":moonSize, "y":moonSize, "z":moonSize},
                "name": "Moon", 
                "texture": "Textures/_k_moon",
                "componentsToAdd": [script2]
            }



# make a copy of the generic dictionary for objects    
moonObject = jdata['genericObject.json'].copy()

# add the generic component attributes
moonObject.update(jdata['genericRigidBody.json'])
moonObject.update(jdata['genericPointerReceiver.json'])
moonObject.update(moon)

# override the generic component attributes
for c in componentsToModify.keys():
    moonObject[c].update(componentsToModify[c])



############     
   

# this uses a simple prefab
sun = {
        "type": "Prefabs/SunPrefab",
        "position": {"x":-39.0, "y":0, "z":0},
        "scale": {"x":5, "y":5, "z":5},
        "name": "Sun",
        "componentsToAdd": []
    }

# make a copy of the generic dictionary for objects    
sunObject = jdata['genericObject.json'].copy()
sunObject.update(sun)

############ 

tdata = {
        "textField": "8000 miles",
        "color": [255,255,0,255],
        "fontSize": 1.0,
        "wrapText": False
        }


# Earth label
 

labelOffset = {"x":0.0, "y":0.3, "z":0.0}
earthLabelPosition = {"x": earthPosition['x'] + labelOffset['x'],
                "y": earthPosition['y'] + labelOffset['y'],
                "z": earthPosition['z'] + labelOffset['z']
               }

labelScale = 0.2
tdata["textField"] = "Earth"
earthLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": earthLabelPosition,
        "rotation": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Earth",
        "name": "earthLabel"
    }


############ 


labelOffset = {"x":0.0, "y":-0.3, "z":0.0}
earthLabelPosition = {"x": earthPosition['x'] + labelOffset['x'],
                "y": earthPosition['y'] + labelOffset['y'],
                "z": earthPosition['z'] + labelOffset['z']
               }



labelScale = 0.2
earthSizeLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": earthLabelPosition,
        "rotation": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Earth",
        "name": "earthSizeLabel"     
    }




############ 




labelOffset = {"x":0.0, "y":0.3, "z":0.0}
moonLabelPosition = {"x": moonPosition['x'] + labelOffset['x'],
                "y": moonPosition['y'] + labelOffset['y'],
                "z": moonPosition['z'] + labelOffset['z']
               }

tdata["textField"] = "The Moon"
tdata["fontSize"] = 1
tdata["color"] = [0, 255, 255, 0]
labelScale = 0.2
moonLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "rotation": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonLabel"     
    }


############ 

labelOffset = {"x":0.0, "y":-0.3, "z":0.0}
moonLabelPosition = {"x": moonPosition['x'] + labelOffset['x'],
                "y": moonPosition['y'] + labelOffset['y'],
                "z": moonPosition['z'] + labelOffset['z']
               }
tdata["textField"] = "2000 miles"
labelScale = 0.2
moonSizeLabel = {
        "type": "Prefabs/textPrefab",
        "tmp": json.dumps(tdata),
        "position": moonLabelPosition,
        "rotation": {"x": 0.0, "y":0.0, "z": 0.0},
        "scale": {"x": labelScale, "y": labelScale, "z": labelScale},
        "enabled": True,
        "parentName": "Moon",
        "name": "moonSizeLabel"     
    }






############ 



############ 



############ 

###############################################
# we will now create the actual objects based on the simple definitions




In [93]:
earthLabel


{'type': 'Prefabs/textPrefab',
 'tmp': '{"textField": "Earth", "color": [255, 255, 0, 255], "fontSize": 1.0, "wrapText": false}',
 'position': {'x': 0.0, 'y': 0.3, 'z': 1.5},
 'rotation': {'x': 0.0, 'y': 0.0, 'z': 0.0},
 'scale': {'x': 0.2, 'y': 0.2, 'z': 0.2},
 'enabled': True,
 'parentName': 'Earth',
 'name': 'earthLabel'}

In [109]:
olist = [sun, earth, moon, earthLabel, earthSizeLabel, moonLabel, moonSizeLabel] #, sun, earthSizeLabel, moonLabel, moonSizeLabel]
olist

[{'type': 'Prefabs/SunPrefab',
  'position': {'x': -39.0, 'y': 0, 'z': 0},
  'scale': {'x': 5, 'y': 5, 'z': 5},
  'name': 'Sun',
  'componentsToAdd': []},
 {'type': 'Prefabs/moveableSphere',
  'position': {'x': 0.0, 'y': 0.0, 'z': 1.5},
  'scale': {'x': 0.08, 'y': 0.08, 'z': 0.08},
  'name': 'Earth',
  'texture': 'Textures/2k_earth_daymap',
  'componentsToAdd': ['{"name": "simpleRotation", "timeRate": 1.0, "rotationTime": 2.0}']},
 {'type': 'Prefabs/moveableSphere',
  'parent': 'Earth',
  'position': {'x': 2.4, 'y': 0.0, 'z': 1.5},
  'scale': {'x': 0.02, 'y': 0.02, 'z': 0.02},
  'name': 'Moon',
  'texture': 'Textures/_k_moon',
  'componentsToAdd': ['{"name": "simpleOrbit", "moonPosition": {"x": 2.4, "y": 0.0, "z": 1.5}, "orbitalPeriod": 56.0, "timeRate": 1.0, "orbitalScale": 1.0, "synchronousRotation": true}']},
 {'type': 'Prefabs/textPrefab',
  'tmp': '{"textField": "Earth", "color": [255, 255, 0, 255], "fontSize": 1.0, "wrapText": false}',
  'position': {'x': 0.0, 'y': 0.3, 'z': 1.5}

# Create the demo clips

In [286]:
'''


    public bool newPosition = true;
    public bool newScale = true;
    public bool newEulerAngles = true;

'''
m1 = {"name": "Earth",
      "activationConditions": 1,
      "reactivateObject": False,
      "enabled": False}

m2 = {"name": "moonLabel",
      "parentName": "Moon",
      "activationConditions": 0,
      "reactivateObject": False,
      "enabled": False
     }

m3 = {"name": "moonLabel",
      "parentName": "Moon",
      "activationConditions": 1,
      "reactivateObject": True,
      "enabled": True
     }
      
clip1 = {"clipName":"clip1", "audioClipString":"Audio/basketball","timeToEnd":29, "autoAdvance": True, "objectChanges": [m2,m3] }


m4 = {"name": "Moon",
      "parentObject": "[_DYNAMIC]",
      "activationConditions": 0,
      "reactivateObject": False,
      "enabled": False
     }


m5 = {"name": "Moon",
      "parentObject": "[_DYNAMIC]",
      "activationConditions": 1,
      "reactivateObject": True,
      "enabled": True
     }

clip2 = {"clipName":"clip2", "timeToEnd":11, "audioClipString":"Audio/not_in_order1","autoAdvance": True, "objectChanges": [m4, m5] }


rtiny = 0.001;
m6 = {"name": "Earth",
     "activationCondition":0,
      "scale": {"x": rtiny, "y":rtiny, "z": rtiny},
      "reactivateObject": False,
      "newPosition": False,
      "newEulerAngles": False,
      "newScale": True
     }

clip3 = {"clipName":"clip3", "timeToEnd":11, "autoAdvance": True, "objectChanges": [m6] }



rtiny = 0.08;
m7 = {"name": "Earth",
     "activationCondition":0,
      "scale": {"x": rtiny, "y":rtiny, "z": rtiny},
      "reactivateObject": False,
      "newPosition": False,
      "newEulerAngles": False,
      "newScale": True
     }

clip4 = {"clipName":"clip4", "timeToEnd":11, "autoAdvance": True, "objectChanges": [m7] }




clips = [clip1, clip2, clip3, clip4]


clips

[{'clipName': 'clip1',
  'audioClipString': 'Audio/basketball',
  'timeToEnd': 29,
  'autoAdvance': True,
  'objectChanges': [{'name': 'moonLabel',
    'parentName': 'Moon',
    'activationConditions': 0,
    'reactivateObject': False,
    'enabled': False},
   {'name': 'moonLabel',
    'parentName': 'Moon',
    'activationConditions': 1,
    'reactivateObject': True,
    'enabled': True}]},
 {'clipName': 'clip2',
  'timeToEnd': 11,
  'audioClipString': 'Audio/not_in_order1',
  'autoAdvance': True,
  'objectChanges': [{'name': 'Moon',
    'parentObject': '[_DYNAMIC]',
    'activationConditions': 0,
    'reactivateObject': False,
    'enabled': False},
   {'name': 'Moon',
    'parentObject': '[_DYNAMIC]',
    'activationConditions': 1,
    'reactivateObject': True,
    'enabled': True}]},
 {'clipName': 'clip3',
  'timeToEnd': 11,
  'autoAdvance': True,
  'objectChanges': [{'name': 'Earth',
    'activationCondition': 0,
    'scale': {'x': 0.001, 'y': 0.001, 'z': 0.001},
    'reactivateOb

In [287]:
# The moduleDescription is the text information that is NOT used by the application at runtime.
# It is only data that we will reference in the database of modules and editing.

moduleDescription = {
        "jsonFileName": "moonEarthSizes.json",
        "moduleName": "Sizes of the Earth and Moon",
        "description": "A demonstration to show the relative sizes and scales of the Earth-Moon System.",  
        "author": "John Wallin",
        "authorInstitution": "Middle Tennessee State University",
        "dateCreated": time.strftime("%c")
    }


# MediaInfo[ { "id:", "type": 1}
# MediaURL

# sceneData contains information about prefabs and prefab specific data
sceneData = {
        "prefabName": "demoPrefab",
        "specificName": "Sizes of the Earth Moon System",  # this will appear on the navigation window
        "educationalObjectives": ["Describe relative sizes of the Earth-Moon system"],
        "instructions": ["Explore the simulation."],
    
        # specific to the demo prefab
        "introAudio": "",
        "createObjects": True,
        "destroyObjects": True,
        "restoreLights": True,
        "timeToEnd": 500,
        "objects": [],
        "useSunlight": True,
        "clips":clips
}



createScene(moduleDescription, sceneData, olist, verbose=False)

True
moonEarthSizes.json

Writing moonEarthSizes.json




In [196]:
#sceneData["objects"] = olist
#sceneData["objects"] = [sun, earth, moon, earthLabel] #, moonLabel, earthSizeLabel, moonSizeLabel]
#sceneData["objects"] = [ moonLabel]

#sceneData
#moonLabel

#sceneData["objects"] = olist
#print(json.dumps(sceneData, indent=4))    

    # make a copy of the generic scene for the activity file
#    myScene = jdata['genericActivity.json'].copy()

    # apply the module description data ot the sceneObject
#    myScene.update(moduleDescription)
#    myScene.update(sceneData)

In [ ]:
ff

In [288]:
outputFile = "earthMoonSizes.json"
outputFile = "demo10.json"
#concatenateScenes(outputPath, outputFile, ['basketBallDemo.json', 'miniEarthMoon.json', 'earthMoon.json'])

outputPath = ""
concatenateScenes(outputPath, outputFile, ['moonEarthSizes.json'] )
                  #,'miniEarthMoon.json']  
                  # 'earthMoon.json'])


 demo10.json
  
 scene
0
moonEarthSizes.json


In [ ]:
outputPath

In [ ]:
import ephem

sun = ephem.Sun()
moon = ephem.Moon()

date = "2020/01/01 00:00:00"

d = ephem.Date(date)
print(d.tuple())

sun.compute(d)
print(sun.ra, sun.dec)

d = ephem.Date(d + 1)
print(d.tuple())
sun.compute(d)
print(sun.ra, sun.dec)
print(sun.hlong, sun.hlat)
print(sun.earth_distance)

moon.compute(d)
print(moon.hlong, moon.hlat)
print(moon.ra, moon.dec)
print(moon.earth_distance)
print(moon.phase)

In [ ]:
date = "2020/01/01 00:00:00"
d = ephem.Date(date)
for dd in range(0,30):
    
    dnew= ephem.date(d+dd)
    sun.compute(dnew)
    moon.compute(dnew)   
    print(dnew, sun.hlong, sun.hlat, moon.phase)

In [ ]:
help(sun)

In [ ]:
def findKeyDiffs(baseClass, olist):
    # find the differences between two dictionaries
    
    # get the keys for the base class
    baseKeys = list(baseClass.keys())
    
    # find the set of common keys for the objects and the set of unique keys
    commonKeys = []
    uniqueKeys = []
    
    # get the key for each object
    for ii in range(len(olist)):
        cc = olist[i]
        objectKeys = list(cc.keys())
    
        # the union of the keys
        uKeys = list(set(baseKeys + objectKeys))
        
        # the intersection of keys
        iKeys = list(set(baseKeys).intersection(objectKeys))
        
        # find the elements only in one list or the other
        aKeys = list( set(baseKeys)^set(objectKeys))

        commonKeys = commonKeys + uKeys
        uniqueKeys = uniqueKeys + aKeys
        
    
    commonKeys = list(set(commonKeys))
    uniqueKeys = list(set(uniqueKeys))
    
    return commonKeys, uniqueKeys
    
        #for ii in range(len(k1)):
        #    k = k1[ii]
        #    if o1[k] != o2[k]:
        #        print(k)
    
    #


def findDiffs(commonKeys, baseClass, olist):
    
    # for each key
    for c in commonKeys:
        baseValue = baseClass[c]
        
        # are there any differences
        diffs = False
        for i in range(len(olist)):
            newValue = olist[i][c]
            if newValue != baseValue:
                diffs = True


        if diffs:
            print("{0:>15}".format(c), end="")
            for i in range(len(olist)):

                if baseValue != olist[i][c]:

                    print("  X   ", end="")
                else:
                    print("  .   ", end="")

            print()
    print(len(olist))
    
commonKeys, uniqueKeys = findKeyDiffs(earthObject, sortList)   
findDiffs(commonKeys, earthObject, sortList)



